In [1]:
# ensure that lists are read as list and not as strings
import pandas as pd
from tqdm import tqdm # progress apply :)
tqdm.pandas()
from ast import literal_eval
columns_to_convert=["texts","labels","retraction_counts","has_retraction","retract_tokens","has_retraction_label"]
converters = {col: literal_eval for col in columns_to_convert}

#df=pd.read_csv("2025-04-03 PDF import complete.json", converters=converters)


df=pd.read_json("2025-04-03 PDF import complete.json")

df=df[df["has_retraction"]==True]


%load_ext autoreload
%autoreload 2
def d(l):
    pd.set_option('display.max_colwidth', l)

In [54]:

df.loc["Science188"][["texts","retract_tokens"]]

texts             [NEWS, Intellectual disability is a condition,...
retract_tokens    [retracting, retraction, retract, retract, ret...
Name: Science188, dtype: object

In [15]:
df["retract_tokens"].explode().value_counts()[5:]

retract_tokens
refract       136
retracts      105
refraction     76
Retracted      27
Retract        27
             ... 
reiraction      1
r tract         1
detracts        1
de tract        1
rge tracts      1
Name: count, Length: 73, dtype: int64

In [4]:
from MyLib import labelfilter

# Apply the functions to create new columns
df['filtered_texts'] = df.apply(lambda row: labelfilter.filter_texts(row['texts'], row['retraction_counts']), axis=1)
df['neighboring_texts'] = df.apply(lambda row: labelfilter.get_neighboring_texts(row['texts'], row['retraction_counts']), axis=1)
df['texts_until_section_header'] = df.apply(lambda row: labelfilter.get_texts_until_section_header(row['texts'], row['labels'], row['retraction_counts']), axis=1)
df[['texts', 'filtered_texts','neighboring_texts','texts_until_section_header']].apply(lambda col: col.apply(len).mean())

texts                         53.915366
filtered_texts                 2.029992
neighboring_texts              7.689811
texts_until_section_header     4.021364
dtype: float64

In [6]:
df["text"]=df.apply(lambda row: labelfilter.get_neighboring_texts(row['texts'], row['retraction_counts']), axis=1).apply(lambda x: " ".join(x))
df.shape

(2434, 13)

In [4]:
df.head()

,path,folder,journal,texts,labels,retraction_counts,has_retraction,retract_tokens,has_retraction_label,filtered_texts,neighboring_texts,texts_until_section_header,text
Science113,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,[The Germans Trias i Pujol University Hospital...,"[caption, text, text, section_header, text, se...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",1.0,"[retracted, retraction, retraction, retraction...","[text, text, text, section_header, text, text,...",[Praised by presidents as a potential miracle ...,"[virus,for instance on the job at ahospitala s...",[Praised by presidents as a potential miracle ...,"virus,for instance on the job at ahospitala st..."
Science116,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[CAFFE', AURORA, CAFFE', IN DEPTH, Italy'sCovi...","[text, text, text, text, text, section_header,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,[retracted],[text],"[""Whatthey'vedoneisbasicallyseenthese twobranc...","[""Whatthey'vedoneisbasicallyseenthese twobranc...","[""Whatthey'vedoneisbasicallyseenthese twobranc...","""Whatthey'vedoneisbasicallyseenthese twobranch..."
Science119,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[SCIENTIFICPUBLISHING, Elsevier to Editor: Cha...","[section_header, section_header, text, text, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,[retracted],[text],[Withdrawn.Two Medical Hypotheses papers—inclu...,"[Charlton says he is “agnostic”, ""This Article...",[Withdrawn.Two Medical Hypotheses papers—inclu...,"Charlton says he is “agnostic” ""This Article-i..."
Science149,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[FEATURES, THE PREPRINT DILEMMA, Biologists ar...","[text, section_header, text, section_header, t...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,[retracting],"[text, text]","[He didn't worry for long.In a few hours, the ...",[Slavov represents the promise of a movement t...,"[He didn't worry for long.In a few hours, the ...",Slavov represents the promise of a movement th...
Science159,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[VACCINEMYTHS, False: Vaccination can cause au...","[section_header, section_header, text, text, t...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1.0,"[retracted, retracted, retracted]","[text, text, text]","[In1998,U.K.doctorAndrew Wakefieldpublishedast...",[False: Mercury in vaccines acts as a neurotox...,"[In1998,U.K.doctorAndrew Wakefieldpublishedast...",False: Mercury in vaccines acts as a neurotoxi...


In [18]:
general_introduction_retract="""

You are a helpful assistant that categorizes the sentences into two retraction contexts, “ARTICLE”, “OTHER”. OTHER contains all sentences that do not match the ARTICLE definition.
If you refer to relevant context or an historical case, explain why this context matters for the categorization. Do not invent context that is not provided.
Return the ID, the result category and give a short explanation (about 50 characters) for the categorisation in a csv format, with the separator=' <SEP> '.
'ID' <SEP> 'category' <SEP> 'explanation'.

Use the following definitions and examples.

"""

science_def="""# ARTICLE: Code a sentence as "ARTICLE" when it discusses the correction or retraction of scientific articles or research findings.
This includes:
    References to errors or misconduct in published research.
    Sentences about the formal process of retracting a paper.
    Discussions on the impact of retractions on authors or the scientific community.
Examples:
- "We're certainly not retract ing our result" says John Kovac, a cosmologist at the Harvard-Smithsonian Center for Astrophysics" - Discusses refusal to retract a scientific result.
- "CERN scientists haven't forgotten the time they found the (still undiscovered) top quark a few years back and then retracted the claim." - Mentions a claim about the top quark being retracted.
- "However, Iran's rank for citations per paper is only 161 (2), and the country ranks second worldwide for the number of retracted papers (2)." - Highlights Iran's high number of retracted papers.
- "I hope that my letter may persuade you to take another look at this subject and possibly you may see fit to write not a retraction, but an extension or redefinition of your first editorial" - Suggests a retraction or redefinition of an editorial.
- "He also retracts earlier statements in which he assertedthatthescopeof thepatentswouldallowthe RoslinInstitute to preventthe technologyfallingintothe wrong hands." - Indicates retraction of earlier statements about patents.
- "Bythe timethemeetingended on18April,the UnitedStateshadretracted or disowned two position papers" - States the U.S. retracted two position papers.
- "Byrne has been working on identifying errors in human-cancer papers since 2015, when she noticed problems with five papers on gene function in cancer cells." – Describes efforts to correct science.
- "For those who do have to retract a paper because of an honest mistake, a few studies offer encouragement: They suggest that such retractions aren’t so bad for one’s career." – Discusses retractions due to honest mistakes and how they might not be detrimental to one's career
- "An analysis of more than 20,600 biomedical papers published from 1995 to 2014 in 40 journals has found that 4% contain deliberately or accidentally duplicated images." - Highlights the presence of deliberately or accidentally duplicated images in biomedical papers, implying scientific misconduct
- "In one case, early-career researchers who showed that a famous scientist had engaged in extensive self-citation and recycled his own publications were accused of being vigilantes and mounting a witch hunt." - Highlights an example of scientific misconduct.
- "a now-retracted paper in Science reported that gay canvassers could sway voter opinions on same-sex marriage." - discusses a retracted paper

Important: When the words retract or retraction are used in the context of physical retraction, do not code as "ARTICLE"



"""

other_def="""# OTHER: Code a sentence with the "OTHER" context when "retract," "refract," or similar terms refer to meanings unrelated to academic publishing, 
such as physical actions or phenomena. This context includes:
    Physical Movements: Sentences that describe the retraction of muscles or body parts.
    Optical Phenomena: Sentences that discuss the refraction of light or similar physical processes.
    General Usage: Sentences that use "retract" or "refract" in everyday language without any connection to scientific articles or research.
Examples:
- "The retraction of the shoulder blades is essential for maintaining good posture during exercise." - Describes physical movement of shoulder blades.
- "Proper retraction of the abdominal muscles can help stabilize the core during heavy lifting." - Refers to muscle retraction for core stability.
- "The oyster, under these circumstances,at once retracts its sensitive mantle-border; the worms, their cirri." - Discusses an oyster retracting its mantle-border.
- "There's sort of long and protracted debates about what really constitutes an aerosol, how small a particle needs to be to be considered an aerosol" - Talks about prolonged debates, not article retraction.
- "TOn  the  non-retractile blood  clot; suppression  of  the formation of blood serum in some pathological states, by M.  G. Hayem.-Research on caramel in wines." - Discusses a non-retractile blood clot in pathology.
- "The scientist explained how different materials can refract light at varying angles." - Explains how light refracts through different materials.
- "Their presence was made evident by the retraction of the mantles when shadows are passed over them." signifies a physical response to environmental stimuli

"""


#print(general_introduction_retract+science_def+other_def)

In [19]:
general_introduction="""

You are a helpful assistant that categorizes the sentences into the retraction frames, “Mistake”, “Misconduct”, or “None”. 
If you refer to relevant context or an historical case, explain why this context matters for the categorization. Do not invent context that is not provided.
Return the ID, the result category and give a short explanation for the categorisation in a csv format, with the separator=' <SEP> '.
'ID' <SEP> 'category' <SEP> 'explanation'

Use the following definitions and examples.

"""

mistake_def="""# Mistake: Code a text with the "Mistake" frame when it emphasizes the correction of a mistake or error without suggesting bad intentions of authors or reviewers. Also consider sentences that evoke or exemplify honesty, responsibility, and the self-correcting nature of science where mistakes can happen, be explained, excused, and corrected.
Examples:
-	"But in science, mistakes are seldom completely fruitless." - This sentence emphasizes the inevitability of mistakes in science and the self-correcting nature of the scientific process.
-	"Mistakes will happen and science does tend to be self-correcting." - This statement underscores the acknowledgment of mistakes as an inherent aspect of scientific research and highlights the self-correcting nature of science.
-	“It can be very difficult to spot one’s own mistakes.” - the difficulty of spotting one's own mistakes indicates an awareness of potential errors to be corrected.
-	“Byrne has been working on identifying errors in human-cancer papers since 2015, when she noticed problems with five papers on gene function in cancer cells.” – The sentences describe efforts to correct science.
-	“For those who do have to retract a paper because of an honest mistake, a few studies offer encouragement: They suggest that such retractions aren’t so bad for one’s career.” – The sentence discusses retractions due to honest mistakes and how they might not be detrimental to one's career, acknowledging that mistakes can happen and can be rectified.

"""
misconduct_def="""# Misconduct: Code a text with the "Misconduct" frame when it suggests scientists’ bad intentions, ethical lapses, or a lack of ethics, virtues, and research integrity education as a problem of science. Also consider sentences that exemplify misconduct by questionable research practices, such as cherry picking, image duplication, fabricated/fake data, or tortured phrases.
Examples:
-	"Such plagiarism is unethical, and it is a form of misconduct, but scientists are not writers." - This sentence explicitly labels plagiarism as a form of misconduct
-	“An analysis of more than 20,600 biomedical papers published from 1995 to 2014 in 40 journals has found that 4% contain deliberately or accidentally duplicated images." - This sentence highlights the presence of deliberately or accidentally duplicated images in biomedical papers, implying scientific misconduct
-	“As funding for research grants becomes more competitive, the pressure to publish in top journals increases and may lead to an increase in ethical lapses.” - This statement hints at the possibility of scientific misconduct.
-	“It appears from the testimony of her graduate students that Dr. Goodwin’s mentoring of her graduate students included behaviors that could be considered scientific misconduct—namely, pressuring students to conceal research results that disagreed with desired outcomes and urging them to over-interpret data that the students themselves considered to be preliminary and weak.”- The behavior described constitutes scientific misconduct.
-	"Springer Nature is retracting 62 articles published in a special issue of one journal." – Even though, the sentence does not directly mention misconduct, the retraction of 62 articles within only one special issue indicates misconduct.
-	“In one case, early-career researchers who showed that a famous scientist had engaged in extensive self-citation and recycled his own publications were accused of being vigilantes and mounting a witch hunt.” - This sentence highlights an example of scientific misconduct, where a famous scientist engaged in extensive self-citation and recycling of publications, and the researchers who exposed this behavior were unfairly accused.

"""
none_def="""# None: Code the text with "None" when no information about the other framings is presented or if the sentence neutrally balances both frames.
Examples:
-	“Posts and article identifiers from the blog Retraction Watch will be incorporated into a web application ….” This sentence discusses a web application, without explicitly framing retractions in terms of mistake or misconduct.
-	“Researchers are still debating what academia can learn from last month’s political science scandal: a now-retracted paper in Science reported that gay canvassers could sway voter opinions on same-sex marriage.”- This sentence presents a discussion about the lesson academia can learn from a retracted paper without explicitly framing the retraction in terms of mistake or misconduct.
"""



#print(general_introduction+mistake_def+misconduct_def+none_def)

In [12]:
from openai import OpenAI
import openAI_key
client = OpenAI(
  api_key=openAI_key.key,
  organization='org-AJfgAhVwZQxIQbEHbpHA7bxV',
  project='proj_DFvnNMN9WCDyFlNo27pEDg0C',
)

In [13]:
from tqdm import tqdm # progress apply :)
tqdm.pandas()

In [14]:
df.folder.unique()

array(['Data_Science\\Newly DL relevant previously marked Technical problem',
       'Data_Science\\relevant_final',
       'Data_Science\\Not relevant\\No occurance of keyword retract',
       'Data_Science\\Not relevant\\Not relevant Type News',
       'Data_Science\\Not relevant\\Not relevant Type Perspectives and Comments',
       'Data_Science\\Not relevant\\Not relevant Type Perspectives and Comments\\TBD',
       'Data_Nature\\Not relevant', 'Data_Nature\\relevant_final',
       'Data_Nature\\Not relevant\\Not relevant (additional)',
       'Data_Nature\\Not relevant\\Not relevant retraction letters'],
      dtype=object)

In [15]:
df["relevant"]=df["folder"].apply(lambda x: "relevant_final" in x)
df

,path,folder,journal,texts,labels,retraction_counts,has_retraction,retract_tokens,has_retraction_label,filtered_texts,neighboring_texts,texts_until_section_header,text,relevant
Science113,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,[The Germans Trias i Pujol University Hospital...,"[caption, text, text, section_header, text, se...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",1.0,"[retracted, retraction, retraction, retraction...","[text, text, text, section_header, text, text,...",[Praised by presidents as a potential miracle ...,"[virus,for instance on the job at ahospitala s...",[Praised by presidents as a potential miracle ...,"virus,for instance on the job at ahospitala st...",False
Science116,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[CAFFE', AURORA, CAFFE', IN DEPTH, Italy'sCovi...","[text, text, text, text, text, section_header,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,[retracted],[text],"[""Whatthey'vedoneisbasicallyseenthese twobranc...","[""Whatthey'vedoneisbasicallyseenthese twobranc...","[""Whatthey'vedoneisbasicallyseenthese twobranc...","""Whatthey'vedoneisbasicallyseenthese twobranch...",False
Science119,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[SCIENTIFICPUBLISHING, Elsevier to Editor: Cha...","[section_header, section_header, text, text, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,[retracted],[text],[Withdrawn.Two Medical Hypotheses papers—inclu...,"[Charlton says he is “agnostic”, ""This Article...",[Withdrawn.Two Medical Hypotheses papers—inclu...,"Charlton says he is “agnostic” ""This Article-i...",False
Science149,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[FEATURES, THE PREPRINT DILEMMA, Biologists ar...","[text, section_header, text, section_header, t...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,[retracting],"[text, text]","[He didn't worry for long.In a few hours, the ...",[Slavov represents the promise of a movement t...,"[He didn't worry for long.In a few hours, the ...",Slavov represents the promise of a movement th...,False
Science159,Data_Science\Newly DL relevant previously mark...,Data_Science\Newly DL relevant previously mark...,Science,"[VACCINEMYTHS, False: Vaccination can cause au...","[section_header, section_header, text, text, t...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1.0,"[retracted, retracted, retracted]","[text, text, text]","[In1998,U.K.doctorAndrew Wakefieldpublishedast...",[False: Mercury in vaccines acts as a neurotox...,"[In1998,U.K.doctorAndrew Wakefieldpublishedast...",False: Mercury in vaccines acts as a neurotoxi...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nature277,Data_Nature\Not relevant\Not relevant retracti...,Data_Nature\Not relevant\Not relevant retracti...,Nature,"[RETRACTION, In view of the fact that the auth...","[section_header, text, text, text, text, text,...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",1.0,"[RETRACTION, retracting, retract, RETRACTED]","[section_header, text, section_header]","[RETRACTION, In view of the fact that the auth...","[RETRACTED, In view of the fact that the autho...","[RETRACTION, In view of the fact that the auth...",RETRACTED In view of the fact that the authors...,False
Nature366,Data_Nature\Not relevant\Not relevant retracti...,Data_Nature\Not relevant\Not relevant retracti...,Nature,"[Retraction, Quantum problems solved through g...","[section_header, section_header, text, text, t...","[1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,"[Retraction, retracting, retracted]","[section_header, text]","[Retraction, In view of the fact that the auth...","[Nature 532,184-185 (2016), Retraction, Sabrin...","[Retraction, Quantum problems solved through g...","Nature 53

In [17]:
def OpenAI_request(row):
    
    try:
        if "name" in row:
            name=row["name"]
        else:
            name = row.name  # Access the index (name)
        text = row['text']  # Access the text column
        question_text=f"The following text with the ID '{name}' must be classified: \n{text}"


        
        completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": general_introduction_retract+science_def+other_def},
            {"role": "user","content": question_text}])
        
        result=completion.choices[0].message.content.split(" <SEP> ")


    
    except:
        print(f"error with row:\n {row}.")
        result=[row.name,None,None]
    return result

result=redo.progress_apply(OpenAI_request, axis=1)

    

100%|████████████████████████████████████████████████████████████████████████████████| 440/440 [06:34<00:00,  1.11it/s]


In [20]:
result_pretty=result.apply(lambda x: x[0].replace.split("<SEP>") if "<SEP>" in x[0] else x)
result_pretty=result_pretty.apply(lambda x: [i.lstrip("category, explanation \n").strip('"').strip("'").strip(": ```csv\n'") for i in x])
result_pretty=result_pretty.apply(lambda x: [i for i in x if (i!="ID") and (i!="category") and (i!="")])


In [100]:
results_df2=pd.DataFrame(result_pretty.to_list(), columns=['name', 'retraction_context', 'retraction_explanation'])
results_df2["retraction_context"]=results_df2.retraction_context.str.replace('CATEGORY: ARTICLE',"ARTICLE")

In [101]:
results_df2["retraction_context"].unique()

array(['ARTICLE', 'OTHER'], dtype=object)

In [183]:
for i in redo.text:
    print(i,"\n\n")

During the last autumn my son, C. C. Trowbridge, who is now a pupil in the Hopkins Grammar School at New Haven,Conn., and who has for several years devoted much of his leisure time to the shot while it was soaring,and called my attention to the fact that the four outer primaries in each wing were interlocked； that part -of each primary along which the lower margin was cut away lapping over or behind the succeeding primary,which was cut along its anterior or upper margin to permit of this interlocking and crossing of these feathers. This was the condition of the wings when he picked up the bird. The general appearances of the wings were so little altered from their ordinary aspects that the interlocking would ordinarily escape notice. My son suggested that this interlocking has the effect of relieving the muscular action required for the extension of the primaries during long flights, especially in soaring birds,and, further, that it might aid the bird in steering its way while soaring.

In [23]:
results_df2.retraction_context.value_counts()

retraction_context
ARTICLE    412
OTHER       28
Name: count, dtype: int64

In [25]:
d(50)
results_df2.retraction_explanation

0      Discusses findings on axon growth and competit...
1                Discusses retraction of a flawed paper.
2       Discusses retraction of Jan Hendrik Schon's PhD.
3            Discusses retraction of a dismissal letter.
4              Discusses retraction of an ecstasy study.
                             ...                        
435           Discusses retraction of Demerec's results.
436         Does not discuss article retraction context.
437       Discusses appeal for retraction of criticisms.
438          Discusses a retraction in academic context.
439    Discusses retraction in the context of academi...
Name: retraction_explanation, Length: 440, dtype: object

In [245]:
# first run:
results_df=pd.read_json("2025-04-03 Retraction GPT3.json")

In [98]:
results_df2

,retraction_context,retraction_explanation
name,,
Nature1010,ARTICLE,Discusses findings on axon growth and competit...
Nature1017,ARTICLE,Discusses retraction of a flawed paper.
Nature1019,ARTICLE,Discusses retraction of Jan Hendrik Schon's PhD.
Nature1024,ARTICLE,Discusses retraction of a dismissal letter.
Nature1030,ARTICLE,Discusses retraction of an ecstasy study.
...,...,...
Science925,ARTICLE,Discusses retraction of Demerec's results.
Science930,OTHER,Does not discuss article retraction context.
Science952,ARTICLE,Discusses appeal for retraction of criticisms.


In [102]:
try:

    dfx.set_index("name", inplace=True)
except:
    dfx["name"]=dfx.index
    

try:
    results_df2.drop_duplicates(subset="name").set_index("name", inplace=True)
except:
    results_df2["name"]=results_df2.index


dfx.update(results_df2)

In [93]:
dfx=pd.read_json('2025-04-03 full data with GPT4o-mini_response.json')
dfx=dfx.sort_values("folder",ascending=False) # this keeps the "relvant_final"
dfx["name"]=dfx.path.apply(lambda x: x.rsplit("\\",1)[-1].split(".")[0])
dfx.drop_duplicates(subset="name",inplace=True)

In [300]:
#df["name"]=df.index
#dfx=pd.merge(df,results_df,on="name",how="left")
#dfx[dfx["retraction_context"]=="OTHER"].folder.value_counts()


In [103]:
d(50)
redo=dfx[(dfx["retraction_context"]=="ARTICLE") & (dfx["relevant"]!=True) ]
redo.shape
#for i in redo.text[:10]:
    #print(i,"\n\n")


(399, 18)

In [106]:
dfx.to_json("2025-04-03 Retraction GPT4o-mini.json")

In [105]:
#dfx["retraction_context"]=dfx.retraction_context.str.replace('CATEGORY: ARTICLE',"ARTICLE")
dfx.retraction_context.value_counts()

retraction_context
ARTICLE    1391
OTHER      1042
Name: count, dtype: int64

In [356]:
def OpenAI_request(row):
    
    try:
        name = row.name  # Access the index (name)
        text = row['text']  # Access the text column
        question_text=f"The following text with the ID '{name}' must be classified: \n{text}"


        
        completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": general_introduction+mistake_def+misconduct_def+none_def},
            {"role": "user","content": question_text}])
        
        result=completion.choices[0].message.content.split(" <SEP> ")


    
    except:
        print(f"error with row:\n {row}.")
        result=[row.name,None,None]
    return result

result_class=redo.progress_apply(OpenAI_request, axis=1)

    

100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [08:45<00:00,  1.32s/it]


In [357]:
result_pretty=result_class.apply(lambda x: x[0].split("<SEP>") if "<SEP>" in x[0] else x)
result_pretty=result_pretty.apply(lambda x: [i.lstrip("ID, category, explanation'\n").strip('"').strip("'").strip(": ```csv\n'") for i in x])
result_pretty=result_pretty.apply(lambda x: [i for i in x if (i!="ID") and (i!="category") and (i!="")])

In [358]:
result_pretty[result_pretty.apply(lambda x: len(x)>3)].values

array([], dtype=object)

In [359]:
results_df=pd.DataFrame(result_pretty.to_list(), columns=['name', 'category', 'explanation'])



In [368]:
category.category.value_counts()


category
Misconduct    767
Mistake       236
None          147
Name: count, dtype: int64

In [364]:
old=pd.read_csv('2025-02-24 data with GPT4o-mini_response.csv')

In [399]:
old

,name,category,explanation
0,Nature1,NaN,The text describes the return of a dinosaur fo...
1,Nature100,Misconduct,The text emphasizes allegations of scientific ...
2,Nature1002,Mistake,The text discusses historical errors in the ci...
3,Nature1003,Misconduct,The text indicates issues regarding the credib...
4,Nature1006,Misconduct,"The text discusses the actions of Tony Leadon,..."
...,...,...,...
1049,Science645,Misconduct,The text describes a case of scientific miscon...
1050,Science928,NaN,The text discusses the role of controversy and...
1051,Science983,Mistake,The text discusses misstatements and misunders...
1052,Nature1358,Mistake,The text highlights multiple discrepancies in ...


In [367]:
category=pd.concat([old,results_df], ignore_index=True)
category.to_csv('2025-04-03 data with GPT4o-mini_response.csv', index=False)

In [362]:
results.name.str.rstrip("09876543212").value_counts()

name
Nature     1637
Science     795
TEXT          1
TNature       1
Name: count, dtype: int64

In [363]:
dfx["retraction_context"].value_counts()

retraction_context
ARTICLE    1391
OTHER      1042
Name: count, dtype: int64

In [388]:
dfx.reset_index(drop = True, inplace = True)
dfx2=pd.merge(dfx,category,on="name",how="outer")



In [112]:
dfx["retraction_context"].value_counts()

retraction_context
ARTICLE    1391
OTHER      1042
Name: count, dtype: int64

In [108]:
dfx.to_csv('2025-04-03 full data with GPT4o-mini_response.csv', index=False)
dfx.to_json('2025-04-03 full data with GPT4o-mini_response.json', index=False)